# Starting w/ Cleaning Drug Take Back Boxes
### want to get to number of boxes per county for 2018

In [1]:
#set up dependencies
import pandas as pd
import numpy as np

#import original csv
drug_boxes = pd.read_csv("../Original Data Pulls/Drug_Take_Back_Boxes.csv")

#get headers
drug_boxes.head()

,Drug Take-Back Site,Zip Code,County,County Code Number,County Code,FIPS County Code,State FIPS,Phone Number,Hours of Availability,Location 1
0,Washington Township Police Dept.,18013,Northampton,48,48,48,42,(610) 599-7828,Contact location for hours,"1021 Washington Boulevard\nBangor, Pennsylvani..."
1,Curwensville Police Dept.,16833,Clearfield,17,17,17,42,(814) 236-3858,Contact location for hours,"900 Susquehanna Avenue\nCurwensville, Pennsylv..."
2,Philadelphia Police Dept.-14th Police District,19144,Philadelphia,51,51,51,42,(215) 686-3140,Contact location for hours,"43 West Haines Street\nPhiladelphia, Pennsylva..."
3,State Police-Troop R / Honesdale,18431,Wayne,64,64,64,42,NaN,NaN,"RR6, 14 Collan Park\nHonesdale, Pennsylvania\n..."
4,State Police-Troop E / Franklin,16323,Venango,61,61,61,42,NaN,NaN,"6724 US 322\nFranklin, Pennsylvania\n(41.3531,..."


In [2]:
#we want to aggregate by county - so total # of boxes per county
#start by getting list of unique county names
counties = drug_boxes["County"].unique()
counties
#get count of number of counties
len(counties)

68

In [3]:
#use group by to get number of boxes in each county
count_by_county = drug_boxes.groupby("County").count()
count_by_county = count_by_county.reset_index()

In [4]:
#save as df we can use to merge etc
drug_boxes_by_county = count_by_county[["County", "Zip Code"]]
drug_boxes_by_county.rename(columns = {'Zip Code':'Number Boxes'}, inplace = True) 
drug_boxes_by_county.head()
drug_boxes_by_county = drug_boxes_by_county.set_index("County")

C:\Users\megam\Anaconda3\lib\site-packages\pandas\core\frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [5]:
#export to csv
drug_boxes_by_county.to_csv("drug_boxes_by_county.csv")

# Cleaning Help Now Intake Data
### goal is to aggregate by county for year; then look at aggregating by month by county so we can examine trends if we want

In [6]:
hotline = pd.read_csv("../Original Data Pulls/Hotline intakes.csv")
hotline.head()

#change date column so easier to use syntax-wise
hotline = hotline.rename(columns={"Week Begin Date": "date", "Total Intakes":"total_intakes"})

hotline.head()

,date,County Name,County Code,County FIPS Code,Latitude/Longitude,total_intakes,Georeferenced Lat/Long
0,8/12/19,Unknown,0,NaN,NaN,0.0,NaN
1,6/3/19,Chester,15,42029.0,"(39.9748705566, -75.7562649818)",2.0,POINT (-75.7562649818 39.9748705566)
2,11/13/17,Union,60,42119.0,"(40.9647313096, -77.0641580308)",0.0,POINT (-77.0641580308 40.9647313096)
3,2/12/18,Bedford,5,42009.0,"(40.0073753643, -78.4911647381)",0.0,POINT (-78.4911647381 40.0073753643)
4,4/17/17,Blair,7,42013.0,"(40.4855502384, -78.349076871)",1.0,POINT (-78.349076871 40.4855502384)


In [7]:
#split week begin date into month, day and year columns so we can easily aggregate by month and year
hotline["year"]=[d.split('/')[2] for d in hotline.date]
hotline["month"]=[d.split('/')[0] for d in hotline.date]

#confirm split and new columns worked
hotline.head()

,date,County Name,County Code,County FIPS Code,Latitude/Longitude,total_intakes,Georeferenced Lat/Long,year,month
0,8/12/19,Unknown,0,NaN,NaN,0.0,NaN,19,8
1,6/3/19,Chester,15,42029.0,"(39.9748705566, -75.7562649818)",2.0,POINT (-75.7562649818 39.9748705566),19,6
2,11/13/17,Union,60,42119.0,"(40.9647313096, -77.0641580308)",0.0,POINT (-77.0641580308 40.9647313096),17,11
3,2/12/18,Bedford,5,42009.0,"(40.0073753643, -78.4911647381)",0.0,POINT (-78.4911647381 40.0073753643),18,2
4,4/17/17,Blair,7,42013.0,"(40.4855502384, -78.349076871)",1.0,POINT (-78.349076871 40.4855502384),17,4


In [8]:
#get counts by year for each county
#start by figuring out how many years worth of data we have
hotline["year"].unique()


array(['19', '17', '18', '16'], dtype=object)

In [9]:
#filter by year and count # of calls by county for that year.  Save in df
#start by creating series for 2016
hotline_2016 = hotline.loc[hotline["year"] == "16"]
hotline_2016 = hotline_2016.groupby("County Name").total_intakes.sum()
hotline_2016

County Name
Adams               0.0
Allegheny         153.0
Armstrong           4.0
Beaver             30.0
Bedford             6.0
Berks              16.0
Blair              10.0
Bradford            3.0
Bucks              42.0
Butler             13.0
Cambria            18.0
Cameron             1.0
Carbon              3.0
Centre              3.0
Chester            26.0
Clarion             1.0
Clearfield          3.0
Clinton             1.0
Columbia            2.0
Crawford            5.0
Cumberland         14.0
Dauphin            31.0
Delaware           50.0
Elk                 1.0
Erie               21.0
Fayette            20.0
Forest              0.0
Franklin            7.0
Fulton              0.0
Greene              1.0
                  ...  
Lehigh             47.0
Luzerne            21.0
Lycoming            7.0
McKean              2.0
Mercer              7.0
Mifflin             0.0
Monroe             13.0
Montgomery         49.0
Montour             2.0
Northampton        22.0
Nort

In [10]:
#now for 2017
hotline_2017 = hotline.loc[hotline["year"] == "17"]
hotline_2017 = hotline_2017.groupby("County Name").total_intakes.sum()
hotline_2017

County Name
Adams               35.0
Allegheny         1049.0
Armstrong           29.0
Beaver             129.0
Bedford             27.0
Berks              135.0
Blair               41.0
Bradford            10.0
Bucks              254.0
Butler              49.0
Cambria             78.0
Cameron              8.0
Carbon              64.0
Centre              27.0
Chester            169.0
Clarion              6.0
Clearfield          31.0
Clinton              9.0
Columbia            24.0
Crawford            28.0
Cumberland          78.0
Dauphin            193.0
Delaware           299.0
Elk                 12.0
Erie               103.0
Fayette            125.0
Forest               0.0
Franklin            45.0
Fulton               7.0
Greene              22.0
                   ...  
Lehigh             215.0
Luzerne            203.0
Lycoming            34.0
McKean              10.0
Mercer              73.0
Mifflin             13.0
Monroe             103.0
Montgomery         277.0
Montour      

In [11]:
#now for 2018
hotline_2018 = hotline.loc[hotline["year"] == "18"]
hotline_2018 = hotline_2018.groupby("County Name").total_intakes.sum()
hotline_2018

County Name
Adams               27.0
Allegheny         1003.0
Armstrong           22.0
Beaver             121.0
Bedford             12.0
Berks              174.0
Blair               52.0
Bradford            19.0
Bucks              274.0
Butler              62.0
Cambria             80.0
Cameron              2.0
Carbon              62.0
Centre              27.0
Chester            173.0
Clarion              5.0
Clearfield          48.0
Clinton             12.0
Columbia            44.0
Crawford            26.0
Cumberland         115.0
Dauphin            219.0
Delaware           336.0
Elk                 16.0
Erie                91.0
Fayette            105.0
Forest               0.0
Franklin            68.0
Fulton               1.0
Greene              14.0
                   ...  
Lehigh             249.0
Luzerne            288.0
Lycoming            23.0
McKean              14.0
Mercer              68.0
Mifflin             12.0
Monroe             107.0
Montgomery         381.0
Montour      

In [12]:
#and for 2019
hotline_2019 = hotline.loc[hotline["year"] == "19"]
hotline_2019 = hotline_2019.groupby("County Name").total_intakes.sum()
hotline_2019

County Name
Adams               24.0
Allegheny          617.0
Armstrong           28.0
Beaver              86.0
Bedford             10.0
Berks               97.0
Blair               27.0
Bradford            15.0
Bucks              173.0
Butler              59.0
Cambria             41.0
Cameron              1.0
Carbon              40.0
Centre              20.0
Chester            114.0
Clarion              6.0
Clearfield          28.0
Clinton              6.0
Columbia            22.0
Crawford            14.0
Cumberland          69.0
Dauphin            150.0
Delaware           332.0
Elk                 18.0
Erie                65.0
Fayette             64.0
Forest               0.0
Franklin            20.0
Fulton               6.0
Greene              17.0
                   ...  
Lehigh             188.0
Luzerne            203.0
Lycoming            19.0
McKean               3.0
Mercer              41.0
Mifflin             18.0
Monroe              78.0
Montgomery         210.0
Montour      

In [13]:
#since 2016 and 2019 are partial years, only interested in 2017 and 2018.  Merge the two series into a dataframe
calls_by_county = pd.DataFrame({"2017": hotline_2017, "2018": hotline_2018})
calls_by_county.head()

,2017,2018
County Name,,
Adams,35.0,27.0
Allegheny,1049.0,1003.0
Armstrong,29.0,22.0
Beaver,129.0,121.0
Bedford,27.0,12.0


In [14]:
#Now to aggregate for each month, year pair to get totals per month.  Yield a series for each month/year
months = np.arange(1,13,1)
months = months.astype(str)
years = ["16", "17", "18", "19"]

hotline.head()

#try looping through a single year
hotline_17 = hotline.loc[hotline["year"] == "17"] 
months2017 = calls_by_county
a = 0
b = 1
for month in months:
    hotline_month = hotline_17.loc[hotline['month']==month]
    hotline_month = hotline_month.groupby("County Name").total_intakes.sum()
    hotline_month = pd.DataFrame(hotline_month)
    months2017 = pd.merge(months2017, hotline_month, on="County Name")
    months2017 = months2017.rename(columns={"total_intakes_x": a, "total_intakes_y":b })
    a+=1
    b+=1

#rename so months are clear
months2017 = months2017.rename(columns = {1: "Jan17", 2: "Feb17", 3: "Mar17",
                                         4: "Apr17", 5:"May17", 6:"Jun17",
                                         7: "Jul17", 8: "Aug17", 9: "Sep17", 
                                          10: "Oct17", 11: "Nov17", 12: "Dec17"})

months2017.head()


,2017,2018,Jan17,Feb17,Mar17,Apr17,May17,Jun17,Jul17,Aug17,Sep17,Oct17,Nov17,Dec17
County Name,,,,,,,,,,,,,,
Adams,35.0,27.0,3.0,3.0,3.0,2.0,3.0,2.0,5.0,1.0,3.0,4.0,3.0,3.0
Allegheny,1049.0,1003.0,130.0,67.0,116.0,105.0,111.0,79.0,100.0,101.0,76.0,71.0,43.0,50.0
Armstrong,29.0,22.0,8.0,1.0,1.0,1.0,2.0,0.0,1.0,5.0,4.0,4.0,2.0,0.0
Beaver,129.0,121.0,17.0,11.0,12.0,11.0,13.0,9.0,18.0,9.0,9.0,11.0,4.0,5.0
Bedford,27.0,12.0,4.0,0.0,3.0,2.0,2.0,2.0,2.0,6.0,1.0,0.0,5.0,0.0


In [15]:
#now for 2018
hotline_18 = hotline.loc[hotline["year"] == "18"] 
months2018 = calls_by_county
a = 0
b = 1
for month in months:
    hotline_month = hotline_18.loc[hotline['month']==month]
    hotline_month = hotline_month.groupby("County Name").total_intakes.sum()
    hotline_month = pd.DataFrame(hotline_month)
    months2018 = pd.merge(months2018, hotline_month, on="County Name")
    months2018 = months2018.rename(columns={"total_intakes_x": a, "total_intakes_y":b })
    a+=1
    b+=1

#rename so months are clear
months2018 = months2018.rename(columns = {1: "Jan18", 2: "Feb18", 3: "Mar18",
                                         4: "Apr18", 5:"May18", 6:"Jun18",
                                         7: "Jul18", 8: "Aug18", 9: "Sep18", 
                                          10: "Oct18", 11: "Nov18", 12: "Dec18"})

months2018 = months2018[["Jan18", "Feb18", "Mar18", "Apr18", "May18", "Jun18",
                        "Jul18", "Aug18", "Sep18", "Oct18", "Nov18", "Dec18"]]
months2018.head()

,Jan18,Feb18,Mar18,Apr18,May18,Jun18,Jul18,Aug18,Sep18,Oct18,Nov18,Dec18
County Name,,,,,,,,,,,,
Adams,1.0,2.0,4.0,0.0,4.0,3.0,3.0,0.0,2.0,5.0,1.0,2.0
Allegheny,71.0,77.0,99.0,122.0,93.0,58.0,95.0,69.0,57.0,89.0,57.0,116.0
Armstrong,1.0,3.0,3.0,0.0,0.0,2.0,1.0,0.0,2.0,3.0,2.0,5.0
Beaver,11.0,12.0,9.0,15.0,12.0,9.0,7.0,10.0,5.0,9.0,9.0,13.0
Bedford,2.0,0.0,0.0,1.0,2.0,3.0,0.0,1.0,2.0,1.0,0.0,0.0


In [84]:
county_hotline_calls = pd.merge(months2017, months2018, on="County Name")
county_hotline_calls_2018 = county_hotline_calls.drop(columns=['2017', 'Jan17', 'Feb17', 'Mar17', 'Apr17', 'May17', 'Jun17','Jul17', 'Aug17', 'Sep17', 'Oct17', 'Nov17', 'Dec17'])

In [87]:
county_hotline_calls_2018 = county_hotline_calls_2018.reset_index()
county_hotline_calls_2018 = county_hotline_calls_2018.rename(columns= {'County Name': 'County'})
county_hotline_calls_2018 = county_hotline_calls_2018.set_index('County')
county_hotline_calls_2018 = county_hotline_calls_2018.drop(columns=['index'])

county_hotline_calls_2018.head()


,2018,Jan18,Feb18,Mar18,Apr18,May18,Jun18,Jul18,Aug18,Sep18,Oct18,Nov18,Dec18
County,,,,,,,,,,,,,
Adams,27.0,1.0,2.0,4.0,0.0,4.0,3.0,3.0,0.0,2.0,5.0,1.0,2.0
Allegheny,1003.0,71.0,77.0,99.0,122.0,93.0,58.0,95.0,69.0,57.0,89.0,57.0,116.0
Armstrong,22.0,1.0,3.0,3.0,0.0,0.0,2.0,1.0,0.0,2.0,3.0,2.0,5.0
Beaver,121.0,11.0,12.0,9.0,15.0,12.0,9.0,7.0,10.0,5.0,9.0,9.0,13.0
Bedford,12.0,2.0,0.0,0.0,1.0,2.0,3.0,0.0,1.0,2.0,1.0,0.0,0.0


In [17]:
#export to csv
county_hotline_calls_2018.to_csv("county_hotline_calls_2018.csv")

# Looking at Naloxone for overdose
### start by creating a table per county for number of administrations, survivals, and type of drug OD'd on

In [18]:
naloxone = pd.read_csv("original data pulls/overdose_network_data.csv")
list(naloxone.columns)

['Incident ID',
 'Incident Date',
 'Incident Time',
 'Day',
 'Incident County Name',
 'Incident State',
 'Victim ID',
 'Gender Desc',
 'Age Range',
 'Race',
 'Ethnicity Desc',
 'Victim State',
 'Victim County',
 'Accidental Exposure',
 'Victim OD Drug ID',
 'Susp OD Drug Desc',
 'Naloxone Administered',
 'Administration ID',
 'Dose Count',
 'Dose Unit',
 'Dose Desc',
 'Response Time Desc',
 'Survive',
 'Response Desc',
 'Revive Action Desc',
 'Third Party Admin Desc',
 'Incident County FIPS Code',
 'Incident County Latitude',
 'Incident County Longitude',
 'Victim County FIPS Code',
 'Victim County Latitude',
 'Victim County Longitude',
 'Incident County Latitude and Longitude',
 'Victim County Latitude and Longitude']

In [19]:
#keep only columns we want to look at for now
naloxone_summary = naloxone[["Incident Date", "Incident County Name", "Naloxone Administered",
                            "Survive", "Susp OD Drug Desc"]]
#rename so we have easier handle to work with for syntax
naloxone_summary = naloxone_summary.rename(columns={"Incident Date": "date",
                                                    "Incident County Name": "County",
                                                    "Naloxone Administered": "naloxone",
                                                    "Susp OD Drug Desc": "od_drug_name"})
naloxone_summary.head()

,date,County,naloxone,Survive,od_drug_name
0,1/4/18,Delaware,Y,N,COCAINE/CRACK
1,1/4/18,Delaware,Y,N,COCAINE/CRACK
2,1/4/18,Delaware,Y,N,HEROIN
3,1/4/18,Delaware,Y,N,HEROIN
4,1/26/18,Chester,N,N,HEROIN


In [20]:
#next, we want to filter by date so we can tally by month
#break down date into month and year
naloxone_summary["year"]=[d.split('/')[2] for d in naloxone_summary.date]
naloxone_summary["month"]=[d.split('/')[0] for d in naloxone_summary.date]

#confirm split and new columns worked
naloxone_summary.head()

,date,County,naloxone,Survive,od_drug_name,year,month
0,1/4/18,Delaware,Y,N,COCAINE/CRACK,18,1
1,1/4/18,Delaware,Y,N,COCAINE/CRACK,18,1
2,1/4/18,Delaware,Y,N,HEROIN,18,1
3,1/4/18,Delaware,Y,N,HEROIN,18,1
4,1/26/18,Chester,N,N,HEROIN,18,1


In [33]:
#find how many years are in data
naloxone_summary["year"].unique()
#find unique values for naloxone
#naloxone_summary["naloxone"].unique()

array(['18'], dtype=object)

In [22]:
#drop rows so we only have data for 2018
indexYears = naloxone_summary[naloxone_summary['year']=="19"].index
naloxone_summary.drop(indexYears, inplace=True)
naloxone_summary

,date,County,naloxone,Survive,od_drug_name,year,month
0,1/4/18,Delaware,Y,N,COCAINE/CRACK,18,1
1,1/4/18,Delaware,Y,N,COCAINE/CRACK,18,1
2,1/4/18,Delaware,Y,N,HEROIN,18,1
3,1/4/18,Delaware,Y,N,HEROIN,18,1
4,1/26/18,Chester,N,N,HEROIN,18,1
5,1/24/18,Beaver,N,Y,HEROIN,18,1
6,1/15/18,Bucks,N,Y,HEROIN,18,1
7,1/15/18,Bucks,N,Y,HEROIN,18,1
8,1/1/18,Philadelphia,N,Y,HEROIN,18,1
9,1/31/18,Washington,N,N,HEROIN,18,1


In [23]:
#map yes values to 1 and no values to 0
naloxone_summary["naloxone"]= naloxone_summary["naloxone"].map({'N':0, 'Y':1})
naloxone_summary["Survive"]=naloxone_summary["Survive"].map({'N':0, 'Y':1})
naloxone_summary.head()

#loop through data frame to build monthly counts/tally
months = np.arange(2,13,1)
months = months.astype(str)

a = 1
b = 2



#do January outside the loop so there is a df to merge future loop iterations to
months2018a = naloxone_summary.loc[naloxone_summary['month']=="1"]
months2018b = months2018a.groupby("County").naloxone.sum()
months2018 = pd.DataFrame(months2018b)

months2018

for month in months:
    naloxone_month = naloxone_summary.loc[naloxone_summary['month']==month]
    naloxone_month = naloxone_month.groupby("County").naloxone.sum()
    naloxone_month = pd.DataFrame(naloxone_month)
    months2018 = pd.merge(months2018, naloxone_month, on="County")
    months2018 = months2018.rename(columns={"naloxone_x": a, "naloxone_y":b })
    a+=1
    b+=1
    
months2018

,1,2,3,4,5,6,7,8,9,10,11,12
County,,,,,,,,,,,,
Allegheny,15,13,37,56,58,81,79,63,72,44,69,54
Beaver,1,2,5,4,1,7,6,9,1,6,5,5
Berks,5,0,7,7,10,5,10,7,10,6,8,8
Bucks,16,13,19,43,63,39,58,62,70,55,55,48
Cambria,1,2,7,10,13,7,2,12,12,7,7,4
Carbon,0,2,3,16,5,2,2,4,13,2,1,2
Centre,2,1,4,1,1,11,4,4,1,0,0,4
Chester,1,2,26,24,41,35,23,20,27,17,14,29
Cumberland,1,2,2,1,12,5,7,6,3,4,5,2


In [40]:
#rename columns
naloxone2018 = months2018.rename(columns={1: "Jan18", 2: "Feb18", 3: "Mar18", 4: "Apr18",
                                         5: "May18", 6: "Jun18", 7: "Jul18", 8: "Aug18",
                                         9: "Sep18", 10: "Oct18", 11: "Nov18", 12: "Dec18"})
naloxone2018.to_csv("clean data/naloxone_by_county.csv")

In [25]:
#find number of people surviving after Naloxone administration
#create df filtered to just cases where naloxone was administered
survival18 = naloxone_summary.loc[naloxone_summary['naloxone']==1]
survival18.head()

,date,County,naloxone,Survive,od_drug_name,year,month
0,1/4/18,Delaware,1,0.0,COCAINE/CRACK,18,1
1,1/4/18,Delaware,1,0.0,COCAINE/CRACK,18,1
2,1/4/18,Delaware,1,0.0,HEROIN,18,1
3,1/4/18,Delaware,1,0.0,HEROIN,18,1
25,2/13/18,Bradford,1,1.0,HEROIN,18,2


In [26]:
#loop through data frame to build monthly counts/tally
months = np.arange(2,13,1)
months = months.astype(str)

a = 1
b = 2

#do January outside the loop so there is a df to merge future loop iterations to
survival2018a = survival18.loc[survival18['month']=="1"]
survival2018b = survival2018a.groupby("County").naloxone.sum()
survival2018 = pd.DataFrame(survival2018b)

survival2018

for month in months:
    survival_month = survival18.loc[survival18['month']==month]
    survival_month = survival_month.groupby("County").naloxone.sum()
    survival_month = pd.DataFrame(survival_month)
    survival2018 = pd.merge(survival2018, survival_month, on="County")
    survival2018 = survival2018.rename(columns={"naloxone_x": a, "naloxone_y":b })
    a+=1
    b+=1
    
survival2018

,1,2,3,4,5,6,7,8,9,10,11,12
County,,,,,,,,,,,,
Allegheny,15,13,37,56,58,81,79,63,72,44,69,54
Beaver,1,2,5,4,1,7,6,9,1,6,5,5
Bucks,16,13,19,43,63,39,58,62,70,55,55,48
Cambria,1,2,7,10,13,7,2,12,12,7,7,4
Chester,1,2,26,24,41,35,23,20,27,17,14,29
Cumberland,1,2,2,1,12,5,7,6,3,4,5,2
Dauphin,7,11,10,9,19,12,13,20,9,11,2,17
Delaware,52,15,46,36,40,35,45,8,9,32,32,22
Erie,3,3,4,4,6,18,23,18,9,12,13,10


In [39]:
#rename columns for clarity
survival2018 = survival2018.rename(columns={1: "Jan18", 2: "Feb18", 3: "Mar18", 4: "Apr18",
                                         5: "May18", 6: "Jun18", 7: "Jul18", 8: "Aug18",
                                         9: "Sep18", 10: "Oct18", 11: "Nov18", 12: "Dec18"})
survival2018.to_csv("clean data/survival_numbers_after_naloxone.csv")


In [28]:
#next look at which drugs were od'd on for which county
#start with summary table from earlier. take a look at what drugs people od'd on
naloxone_summary["od_drug_name"].unique()


array(['COCAINE/CRACK', 'HEROIN', 'FENTANYL',
       'FENTANYL ANALOG/OTHER SYNTHETIC OPIOID', 'UNKNOWN',
       'PHARMACEUTICAL OPIOID', 'MARIJUANA', 'ALCOHOL',
       'SYNTHETIC MARIJUANA', 'PHARMACEUTICAL OTHER',
       'BENZODIAZEPINES (I.E.VALIUM, XANAX, ATIVAN, ETC)', 'OTHER',
       'BARBITURATES (I.E. AMYTAL, NEMBUTAL, ETC)', 'CARFENTANIL',
       'SUBOXONE', 'METHADONE', 'METHAMPHETAMINE', 'BATH SALTS',
       'PHARMACEUTICAL STIMULANT'], dtype=object)

In [29]:
drug_of_choice = naloxone_summary.loc[naloxone_summary['naloxone']==1]
drug_of_choice.head()

,date,County,naloxone,Survive,od_drug_name,year,month
0,1/4/18,Delaware,1,0.0,COCAINE/CRACK,18,1
1,1/4/18,Delaware,1,0.0,COCAINE/CRACK,18,1
2,1/4/18,Delaware,1,0.0,HEROIN,18,1
3,1/4/18,Delaware,1,0.0,HEROIN,18,1
25,2/13/18,Bradford,1,1.0,HEROIN,18,2


In [38]:
#create new column to map all opiates into a simple 1=yes, 0=no
#most concerned around circumstances surrounding opiates
drug_of_choice["opiate"]=np.where(drug_of_choice["od_drug_name"].str.contains("HEROIN")| 
                                  drug_of_choice["od_drug_name"].str.contains("FENTANYL")|
                                  drug_of_choice["od_drug_name"].str.contains("PHARMACEUTICAL OPIOID")|
                                  drug_of_choice["od_drug_name"].str.contains("CARFENTANIL")|
                                  drug_of_choice["od_drug_name"].str.contains("SUBOXONE")|
                                  drug_of_choice["od_drug_name"].str.contains("METHADONE"), 1, 0)
drug_of_choice.to_csv("clean data/naloxone_cleaned.csv")

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [36]:
#find total number of opiate related nalaxone administrations by county
opiate_county = drug_of_choice.groupby("County").opiate.sum()
opiate_county = pd.DataFrame(opiate_county)
opiate_county

,opiate
County,
Adams,21
Allegheny,517
Armstrong,14
Beaver,43
Bedford,11
Berks,66
Blair,17
Bradford,6
Bucks,485


In [37]:
#export to csv
opiate_county.to_csv("clean data/opiates_and_naloxone_bycounty.csv")